# 1. install libaries

In [20]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install XBRL

Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install oauth2

     |████████████████████████████████| 102kB 8.1MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
conda install -c anaconda openssl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


# 2. import libraries

In [2]:
import requests

In [3]:
import os, re, sys, json

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import getpass

# 3. Authentication for xbrl token

### 3.1. acess token

In [7]:
"""
get access token from XBRL API
"""
# print('Enter XBRL US Web account email: ')
email = 'jesse.c.jia@gmail.com'
# print('Account password: ')
password = 'QAZwsx123'
# print('Client ID: ')
clientid = 'bf3ba33f-e98b-4159-aea2-466445f9621f'
# print('Client secret: ')
secret = 'a3f87777-0086-4208-8d3b-df494408271e'

body_auth = {'username' : ''.join(email), 
            'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'password' : ''.join(password), 
            'grant_type' : 'password', 
            'platform' : 'ipynb' }
endpoint_auth = 'https://api.xbrl.us/oauth2/token'
res = requests.post(endpoint_auth, data=body_auth)
auth_json = res.json()

if 'error' in auth_json:
        print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
        print ("\n\nYour access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. \n\nFor now, skip ahead to the section 'Make a Query'.")

access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''
    
    



Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.



### 3.2. refresh token

In [54]:

"""
reconnect to XBRL API when expire
"""
token = token if newrefresh != '' else refresh_token 

refresh_auth = {'client_id': ''.join(clientid), 
            'client_secret' : ''.join(secret), 
            'grant_type' : 'refresh_token', 
            'platform' : 'ipynb', 
            'refresh_token' : ''.join(token) }
refreshres = requests.post(endpoint_auth, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)

Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.
58b45f0e-6fc4-4e20-978d-bdb4cec7e5ab


# 4. Prepare Data

### 4.1 file and data path

In [8]:
"""
get document path
"""
root_path = os.path.abspath(os.path.dirname(os.getcwd()))
data_path = os.path.join(root_path, 'data')
company_path = os.path.join(data_path, 'company')
company_cik_path = os.path.join(company_path, 'CIK_Summary.csv')
sec_cfp_path = os.path.join(data_path, "SEC_FIN_data")
sec_argriculture = os.path.join(sec_cfp_path, "agriculture")


sic100_file_path = os.path.join(sec_cfp_path, "SIC100")
sic100_df_path = os.path.join(sic100_file_path, "SIC100CompanyList.xlsx")

sic200_file_path = os.path.join(sec_cfp_path, "SIC200")
sic200_df_path = os.path.join(sic200_file_path, "SIC200CompanyList.xlsx")

sic800_file_path = os.path.join(sec_cfp_path, "SIC800")
sic800_df_path = os.path.join(sic800_file_path, "SIC800CompanyList.xlsx")

sic700_file_path = os.path.join(sec_cfp_path, "SIC700")
sic700_df_path = os.path.join(sic700_file_path, "SIC700CompanyList.xlsx")

sic2870_file_path = os.path.join(sec_cfp_path, "SIC2870")
sic2870_df_path = os.path.join(sic2870_file_path, "SIC2870CompanyList.xlsx")

In [9]:
sic100_df_path

'/Users/j/Documents/GitHub/MAST90106/data/SEC_FIN_data/SIC100/SIC100CompanyList.xlsx'

### 4.2. read cik file

In [81]:
"""
import company-cik file
"""

#read cik.csv as a pandas DataFrame
ciksDF = pd.read_csv(company_cik_path, header=None)

#set the column with all the tickers as the index
ciksDF = ciksDF.set_index(0)


### 4.3. define company name and cik converting function

In [82]:
#function to return CIK of a ticker
def getCIK(ticker):
    """
    input ticker
    return respect cik
    """
    try:
        return int(ciksDF.loc[ticker.lower(), 1])

    #return None there's no CIK for the ticker
    except:
        print(f"{ticker} does not exist.")
        return None
    
    
#function to return CIK of a list of tickers
def outputcik(TICKERS): 
    """
    input ticker list
    return cik repsectively in list
    """
    cikDir = []
    for ticker in TICKERS:
        a = str(getCIK(ticker))
        if a != 'None':
            cik = a.zfill(10)
            cikDir.append(cik)
        else:
            pass
    return cikDir

In [83]:
# test code
lis = outputcik(["uytuyfd","a"])
lis

uytuyfd does not exist.


['0001090872']

### 4.4. Read Company List

In [84]:
# company list

company_list = ["YTEN", "UAN", "CGA", "RKDA", "FMC", "MOS", "SEED", "CTVA", "MBII", "AVD", "ICL", "IPI", "NTR", "MGPI", "SMG", "CF"]

In [85]:
len(company_list)

16

### 4.5. convert company_name into cik

In [86]:
# generate cik list
cik_list = outputcik(company_list)

In [87]:
# cik_list1 = cik_list.pop(2)

In [88]:
len(cik_list)

16

In [89]:
cik_list = (cik_list)

### 4.6. financial data conceptual name

In [64]:
# financial data with the conceptual name on SEC website respectively
Financial_data_dic = {
    "revenue" : "RevenueFromContractWithCustomerExcludingAssessedTax",
    "net_income" : "NetIncomeLoss",
    "operating_income" : "OperatingIncomeLoss", 
    "asset" : "Assets",
    "current_asset" : "AssetsCurrent",
    "inventory" : "InventoryNet",
    "current_liability" : "LiabilitiesCurrent",
    "debt" : "LongTermDebtNoncurrent",
    "equity" : "StockholdersEquity",
    "eps" : "EarningsPerShareBasic"
}

### 4.7. read cfp file function

In [18]:
def read_fin_data_file(path):
    df = pd.read_csv(path)
    df = df.set_index('Unnamed: 0')
    return df

### 4.80 process sic

In [19]:
def SIC_convert(sic, path):
    df = pd.read_excel(path)
    df = df.drop(df[df['State/Country'].isin(states) == False].index)
    l = list(df['CIK'])
    print(l)
    cik_list = []
    count = 0
    for cik in l:
        cik = str(cik)
        cik = cik.zfill(10)
        cik_list.append(cik)
        count += 1
        print(count)
    return cik_list

### 4.8 SIC company list

In [65]:
sic100 = SIC_convert('100', sic100_df_path)

[1129641, 876320, 842888, 1650505, 3545, 1519894, 1807707, 1572819, 715374, 1610462, 1011134, 793931, 806179, 1290504, 943941, 767884, 1501958, 101063, 1483752, 1522155, 1064047, 1776738, 1755672, 835015, 902277, 730985, 18169, 1065188, 1614924, 1852776, 1169745, 1047340, 1796412, 1780914, 1282001, 1816814, 740124, 1795139, 1818495, 1003619, 1003515, 760612, 1002771, 1840780, 1794009, 1342423, 1838128, 726293, 818010, 1793341, 824103, 1054476, 1222557, 78716, 1657737, 1051869, 792161, 1477246, 1043902, 1039213, 1078259, 1269942, 1086313, 707600, 1822966, 861291, 1599811, 1876945, 1853537, 1302946, 1669400, 1280396, 1123312, 1548240, 1761475, 1803705]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


In [66]:
sic200 = SIC_convert('200', sic200_df_path)

[1331427, 927536, 1023367, 16160, 1096662, 34551, 1440517, 1418115, 774415, 1000778, 811930, 1425905, 1138659, 1756180, 1118650, 1116684, 882991, 1499790, 724915, 729499, 1072287, 1289237, 819215]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [67]:
sic700 = SIC_convert('700', sic700_df_path)

[1043512, 1800637, 1531088, 1729481, 878944, 1043513, 1020866, 1216018, 1734713, 1133470, 1376793, 1025771, 1042698, 277638, 1197115, 1043516, 29006, 876103, 1573590, 1540334, 1489874, 1424264, 1472998, 1497055, 937601, 1057949, 1802974, 813742, 1671132, 1043517, 1331931, 931612, 1059975, 1463191, 1043519, 1729751, 1439813, 1042695, 1043523, 1043524, 1042694, 1043526, 1042327, 1042696, 1042697, 1482541, 831002, 817366, 1349302, 1574186]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [68]:
sic800 = SIC_convert('800', sic800_df_path)

[824430, 1497288, 795255, 1049484, 1049486, 1134513, 1047739, 761860, 1049479, 1766280, 63814, 1049481, 1111118, 1543265, 784011, 778439, 1049475, 1066797, 1049478, 1043012, 1045466, 1140868, 1687316]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [69]:
sic2870 = SIC_convert('2870', sic2870_df_path)

[47968, 1369608, 1014455, 1592016, 708484, 1031896, 5981, 862450, 883906, 1162141, 1162149, 875729, 883076, 882483, 1705843, 1488435, 1324404, 1394920, 1162145, 1484839, 1026601, 1071918, 1425292, 319124, 1525998, 814050, 882259, 1081158, 36537, 1511357, 1046340, 880882, 1100976, 793160, 825203, 1046337, 1159275, 745394, 1441693, 66895, 1110783, 1324087, 1071916, 1285785, 820626, 1160701, 70083, 832975, 1046338, 840759, 1517248, 793421, 1444754, 1663453, 1162144, 1575858, 868725, 1789192, 825542, 1680378, 1071919, 1505409, 1162140, 1478513, 1394921, 722079, 1162147, 1162148, 1477600, 1162146, 1394919, 879575, 1337606, 1162137, 1162142, 1279529, 862612, 1083200, 1097805, 1071917, 1435224, 866752, 873728, 1046339, 750150]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85


In [90]:
cik_list = cik_list + sic100 + sic200 + sic2870 + sic700 + sic800

In [93]:
cik_list = list(set(cik_list))

In [94]:
len(cik_list)

266

In [28]:
cik_tuple = []
for c in cik_tu:
    c = str(c)
    cik_tuple.append(c)
tuple(cik_tuple)

NameError: name 'cik_tu' is not defined

### 4.10 US state list

In [29]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# 5. Request data via XBRL

In [30]:
def acquire_data(company_cik, financial_data):
    """
    input company cik value and financial conceptual name on SEC
    return data frame of financial value, year, entity name, and so on, (details see fiels list)
    """
    
    # Define the fields to be returned
    fields = ['period.fiscal-year.sort(DESC)',
              'period.end.sort(DESC)',
              'entity.name.sort(ASC)',
              'entity.cik.sort(ASC)',
              'concept.local-name.sort(ASC)',
              'fact.value',
              'unit',
              'report.filing-date']
    
    # Filter the query for specific XBRL elements (tags) to research
    XBRL_Elements = [financial_data
                ]
    
    # Filter companies for the query
    companies_cik = [company_cik
                ]
    
    # api parameters
    params = {'fields': ','.join(fields),
         'concept.local-name': ','.join(XBRL_Elements),
         'entity.cik': ','.join(companies_cik),
         #'period.fiscal-year': ','.join(years),
         'period.fiscal-period': 'Y',
         'fact.has-dimensions': 'FALSE',
         'fact.ultimus': 'TRUE'
         }
    
    search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
    res_json = res.json()
    res_df = pd.DataFrame(res_json['data'])

    
#     print(res_json['paging'])
#     print(res_df)
    return res_df

In [31]:
# TEST CODE
df_test = acquire_data('0001121702',"Assets")

In [32]:
df_test

,period.fiscal-year,period.end,entity.name,entity.cik,concept.local-name,fact.value,unit,report.filing-date
0,2020,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,14522000,USD,2021-08-11
1,2019,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,16718000,USD,2021-03-16
2,2018,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,12960000,USD,2020-03-25
3,2017,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,16882000,USD,2019-03-28
4,2016,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,10748000,USD,2018-03-12
5,2015,None,"YIELD10 BIOSCIENCE, INC.",0001121702,Assets,17088000,USD,2017-03-30
6,2014,None,"METABOLIX, INC.",0001121702,Assets,23135000,USD,2016-03-29
7,2013,None,"METABOLIX, INC.",0001121702,Assets,26738000,USD,2015-03-25
8,2012,None,"METABOLIX, INC.",0001121702,Assets,53510000,USD,2014-03-28
9,2011,None,"METABOLIX, INC.",0001121702,Assets,82912000,USD,2013-03-28


In [33]:
# for cik in cik_list:
#     #print(cik)
#     cik_fin_value_df = df_empty = pd.DataFrame(columns=['A', 'B', 'C', 'D'])
#     for k, v in Financial_data_dic.items():
#         print(cik,v)
#         temp_df = acquire_data(cik, v) asset
#         print(temp_df)
        




In [34]:
def generate_financial_data(cik, Financial_data_dic):
    """
    input company's cik value and financial item dic
    acquire financial value through xrbl API,
    calculate financial ratio and append in data frame
    return a data frame with year, company name, financial ratios
    """
    
    # create a empty data frame for use
    cik_fin_value_df = pd.DataFrame()
    # cik = "0001121702"
    for k, v in Financial_data_dic.items():
        print(f"current cik is {cik}, acquire {k}")
        
        # return first financial values into df
        if k == "revenue":
            
            cik_fin_value_df = acquire_data(cik, v)
            if cik_fin_value_df.empty:
                pass
            else:
                cik_fin_value_df.rename(columns = {"fact.value" : k}, inplace = True)
                cik_fin_value_df[k] = pd.to_numeric(cik_fin_value_df[k])
                cik_fin_value_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "unit", axis = 1, inplace = True)
                cik_fin_value_df.drop(labels = "report.filing-date", axis = 1, inplace = True)

    #         cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)

        
        # create temp df, and acquire rest financial data, then concate with inital df
        else:
#             print(cik,k,v)
            temp_df = acquire_data(cik, v)
            if temp_df.empty:
                temp_df[k] = np.NaN
                cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
            else:
                temp_df.rename(columns = {"fact.value" : k}, inplace = True)
                temp_df[k] = pd.to_numeric(temp_df[k])

                temp_df.drop(labels = "concept.local-name", axis = 1, inplace = True)
    #             temp_df.drop(labels = "period.end", axis = 1, inplace = True)
                temp_df.drop(labels = "unit", axis = 1, inplace = True)
                temp_df.drop(labels = "report.filing-date", axis = 1, inplace = True)
    #             temp_df.drop(labels = "entity.name", axis = 1, inplace = True)


    #             if len(temp_df["period.end"]) >= len(cik_fin_value_df["period.end"]):
    #                 cik_fin_value_df.drop(labels = "period.end", axis = 1, inplace = True)
    #             else:
                temp_df.drop(labels = "period.end", axis = 1, inplace = True)
    
                # keep the most company name column
                try:
                    if len(temp_df["entity.cik"]) >= len(cik_fin_value_df["entity.cik"]):
                        cik_fin_value_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        cik_fin_value_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                    else:
                        temp_df.drop(labels = "entity.name", axis = 1, inplace = True)
                        temp_df.drop(labels = "entity.cik", axis = 1, inplace = True)
                except:
                    pass



    #             cik_fin_value_df = pd.concat([cik_fin_value_df, temp_df], axis = 1)
    #             cikabs_fin_value_df = pd.concat((cik_fin_value_df, temp_df), axis = 1, join = "outer")
                try:
                    cik_fin_value_df = pd.merge(cik_fin_value_df, temp_df, on = "period.fiscal-year", how = "outer")
                except:
                    pass
    
    # calculate financial ratios
#     Ratios_calculator(cik_fin_value_df)
    print(f"{cik} fin data acquire compeleted.")
    save_csv(cik_fin_value_df, sec_argriculture, cik)
    
    return cik_fin_value_df



# cik_fin_value_df["ROA1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)
# cik_fin_value_df["ROE1"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)
# cik_fin_value_df["ROC1"] = cik_fin_value_df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)
# cik_fin_value_df["OperatingMargin"] = cik_fin_value_df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["NetMargin"] = cik_fin_value_df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
# cik_fin_value_df["CurrentRatio"] = cik_fin_value_df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)
# cik_fin_value_df["QuickRatio"] = cik_fin_value_df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)
# cik_fin_value_df["DERatio"] = cik_fin_value_df.apply(lambda x:x["debt"]/x["equity"], axis = 1)



In [35]:
# TEST CODE

# test1_df = generate_financial_data('0000857949', Financial_data_dic)

In [36]:
def ROA1(df):
    df["ROA1"] = df.apply(lambda x:x["net_income"]/x["asset"], axis = 1)

def ROE1(df):
    df["ROE1"] = df.apply(lambda x:x["net_income"]/x["equity"], axis = 1)

def ROC1(df):
    df["ROC1"] = df.apply(lambda x:(x["net_income"] + x["debt"])/x["asset"], axis = 1)

def Op_Margin(df):
    df["OperatingMargin"] = df.apply(lambda x:x["operating_income"]/x["revenue"], axis = 1)
    
def Net_Margin(df):
    df["NetMargin"] = df.apply(lambda x:x["net_income"]/x["revenue"], axis = 1)
    
def Current_Ratio(df):
    df["CurrentRatio"] = df.apply(lambda x:x["current_asset"]/x["current_liability"], axis = 1)

def Quick_Ratio(df):
    df["QuickRatio"] = df.apply(lambda x:(x["current_asset"]-x["inventory"])/x["current_liability"], axis = 1)

def DE(df):
    df["DERatio"] = df.apply(lambda x:x["debt"]/x["equity"], axis = 1)


In [37]:
def Ratios_calculator(df):
    
    for k, v in Financial_data_dic.items():
        df[k] = pd.to_numeric(df[k])
    ROA1(df)
    ROE1(df)
    ROC1(df)
    Op_Margin(df)
    Net_Margin(df)
    Current_Ratio(df)
    Quick_Ratio(df)
    DE(df)
    
    return df

In [99]:
def generate_financial_ratio(cik, industry, valid_list, invalid_list, count):
#     generate_financial_data(cik, fin_data_dic)

    df = pd.DataFrame()
    try:
        df = read_fin_data_file(f'{sec_cfp_path}/{industry}_comprehensive/{cik}.csv')
        print(1)
        Ratios_calculator(df)
        print(f"{cik} fin ratios calculated")
        df.to_csv(f"{sec_cfp_path}/{industry}_individual_with_ratios/{cik}.csv")
        count += 1
        valid_list.append(cik)
    except:
        print(f"{cik} has no ratios return")
        invalid_list.append(cik)
    


#     try:
#         Ratios_calculator(df)
#         #print(df)
#         print(f"{cik} fin ratios calculated")
#         valid_list.append(cik)
#     except:
#         print(f"{cik} has no ratios return")
#         invalid_list.append(cik)
#         pass
    
    return df, valid_list, invalid_list, count

In [100]:
def output_fin_data_file(cik_list, fin_data_dic, industry):
    
    valid_company_list = []
    invalid_company_list = []
    path = os.path.join(sec_cfp_path, industry)

    for c in cik_list:
        df = generate_financial_data(c, fin_data_dic)
        print("df is here")
        print(df)
        save_csv(df, path, c)
    
    print(f"The list cik companies have connection with SEC website:")
    for cik in valid_company_list:
        print(cik)

    print(f"The list cik companies have no connection with SEC website, no return data via XBRL:")
    for cik in invalid_company_list:
        print(cik)
    
    return None

In [101]:
def output_file(cik_list, fin_data_dic, industry):
    industry_df = pd.DataFrame()
    
    valid_company_list = []
    invalid_company_list = []
    count = 0

    for c in cik_list:
        count+=1
        print(count)
#         generate_financial_data(c, fin_data_dic)
        temp_df, valid_company_list, invalid_company_list, count = generate_financial_ratio(c, industry, valid_company_list, invalid_company_list, count)
        #temp_df.to_csv(f"{sec_cfp_path}/{industry}_individual_with_ratios/{c}.csv")
        industry_df = pd.concat([industry_df, temp_df], axis = 0)
        
#         print(f"{c} has been added.")
    
    industry_df.to_csv(f"{sec_cfp_path}/{industry}.csv")
    
#     print(f"The list cik companies have connection with SEC website: {valid_company_list}.")
#     print(f"The list cik companies have no connection with SEC website, no value returned by using XBEL: {valid_company_list}.")
    print(f'There are {count} companies in {industry} industry.')
    return industry_df, valid_company_list, invalid_company_list

# 6. output process

### 6.1. define save function

In [48]:
def save_csv(file, path, industry):
    file.to_csv(f"{path}/{industry}.csv")
    print(f"{file} has been saved as csv.")
    return None

### 6.2. output financial data into single file for each compnay

In [56]:
output_fin_data_file(sic2870, Financial_data_dic, "SIC2870")

current cik is 0000047968, acquire revenue
current cik is 0000047968, acquire net_income
current cik is 0000047968, acquire operating_income
current cik is 0000047968, acquire asset
current cik is 0000047968, acquire current_asset
current cik is 0000047968, acquire inventory
current cik is 0000047968, acquire current_liability
current cik is 0000047968, acquire debt
current cik is 0000047968, acquire equity
current cik is 0000047968, acquire eps
0000047968 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001369608, acquire revenue

current cik is 0000708484, acquire net_income
current cik is 0000708484, acquire operating_income
current cik is 0000708484, acquire asset
current cik is 0000708484, acquire current_asset
current cik is 0000708484, acquire inventory
current cik is 0000708484, acquire current_liability
current cik is 0000708484, acquire debt
current cik is 0000708484, acquire equity
current cik is 0000708484, acquire eps
0000708484 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001031896, acquire revenue
current cik is 0001031896, acquire net_inc

current cik is 0000862450, acquire net_income
current cik is 0000862450, acquire operating_income
current cik is 0000862450, acquire asset
current cik is 0000862450, acquire current_asset
current cik is 0000862450, acquire inventory
current cik is 0000862450, acquire current_liability
current cik is 0000862450, acquire debt
current cik is 0000862450, acquire equity
current cik is 0000862450, acquire eps
0000862450 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0000883906, acquire revenue
current cik is 0000883906, acquire net_inc

current cik is 0001488435, acquire net_income
current cik is 0001488435, acquire operating_income
current cik is 0001488435, acquire asset
current cik is 0001488435, acquire current_asset
current cik is 0001488435, acquire inventory
current cik is 0001488435, acquire current_liability
current cik is 0001488435, acquire debt
current cik is 0001488435, acquire equity
current cik is 0001488435, acquire eps
0001488435 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001324404, acquire revenue
current cik is 0001324404, acquire net_inc

current cik is 0000319124, acquire net_income
current cik is 0000319124, acquire operating_income
current cik is 0000319124, acquire asset
current cik is 0000319124, acquire current_asset
current cik is 0000319124, acquire inventory
current cik is 0000319124, acquire current_liability
current cik is 0000319124, acquire debt
current cik is 0000319124, acquire equity
current cik is 0000319124, acquire eps
0000319124 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001525998, acquire revenue
current cik is 0001525998, acquire net_inc

...

current cik is 0001663453, acquire net_income
current cik is 0001663453, acquire operating_income
current cik is 0001663453, acquire asset
current cik is 0001663453, acquire current_asset
current cik is 0001663453, acquire inventory
current cik is 0001663453, acquire current_liability
current cik is 0001663453, acquire debt
current cik is 0001663453, acquire equity
current cik is 0001663453, acquire eps
0001663453 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001162144, acquire revenue
current cik is 0001162144, acquire net_inc

current cik is 0000868725, acquire net_income
current cik is 0000868725, acquire operating_income
current cik is 0000868725, acquire asset
current cik is 0000868725, acquire current_asset
current cik is 0000868725, acquire inventory
current cik is 0000868725, acquire current_liability
current cik is 0000868725, acquire debt
current cik is 0000868725, acquire equity
current cik is 0000868725, acquire eps
0000868725 fin data acquire compeleted.
Empty DataFrame
Columns: []
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: [] has been saved as csv.
current cik is 0001789192, acquire revenue
current cik is 0001789192, acquire net_income
current cik is 0001789192, acquire operating_income
current cik is 0001789192, acquire asset
current cik is 0001789192, acquire current_asset
current cik is 0001789192, acquire inventory
current cik is 0001789192, acquire current_liability
current cik is 0001789192, acquire debt
current cik i

current cik is 0001680378, acquire net_income
current cik is 0001680378, acquire operating_income
current cik is 0001680378, acquire asset
current cik is 0001680378, acquire current_asset
current cik is 0001680378, acquire inventory
current cik is 0001680378, acquire current_liability
current cik is 0001680378, acquire debt
current cik is 0001680378, acquire equity
current cik is 0001680378, acquire eps
0001680378 fin data acquire compeleted.
Empty DataFrame
Columns: [eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [eps]
Index: []
Empty DataFrame
Columns: [eps]
Index: [] has been saved as csv.
current cik is 0001071919, acquire revenue
current cik is 0001071919, acquire net_income
current cik is 0001071919, acquire operating_income
current cik is 0001071919, acquire asset
current cik is 0001071919, acquire current_asset
current cik is 0001071919, acquire inventory
current cik is 0001071919, acquire current_liability
current cik is 0001071919, acquire debt
curr

current cik is 0001477600, acquire net_income
current cik is 0001477600, acquire operating_income
current cik is 0001477600, acquire asset
current cik is 0001477600, acquire current_asset
current cik is 0001477600, acquire inventory
current cik is 0001477600, acquire current_liability
current cik is 0001477600, acquire debt
current cik is 0001477600, acquire equity
current cik is 0001477600, acquire eps
0001477600 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001162146, acquire revenue
current cik is 0001162146, acquire net_inc

current cik is 0001083200, acquire net_income
current cik is 0001083200, acquire operating_income
current cik is 0001083200, acquire asset
current cik is 0001083200, acquire current_asset
current cik is 0001083200, acquire inventory
current cik is 0001083200, acquire current_liability
current cik is 0001083200, acquire debt
current cik is 0001083200, acquire equity
current cik is 0001083200, acquire eps
0001083200 fin data acquire compeleted.
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
df is here
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: []
Empty DataFrame
Columns: [net_income, operating_income, asset, current_asset, inventory, current_liability, debt, equity, eps]
Index: [] has been saved as csv.
current cik is 0001097805, acquire revenue
current cik is 0001097805, acquire net_inc

### 6.3. generate cfp in one file|

In [102]:
indus, valid_company_list, invalid_company_list = output_file(cik_list, Financial_data_dic, "agriculture")

1
0001046338 has no ratios return
2
0001129641 has no ratios return
3
0001000778 has no ratios return
4
0001118650 has no ratios return
5
0000820626 has no ratios return
6
0001818495 has no ratios return
7
0001840780 has no ratios return
8
0001162147 has no ratios return
9
0001025771 has no ratios return
10
0001599811 has no ratios return
11
0001071918 has no ratios return
12
0000722079 has no ratios return
13
1
0000003545 fin ratios calculated
15
1
0001756180 fin ratios calculated
17
0001014455 has no ratios return
18
0001394919 has no ratios return
19
0001766280 has no ratios return
20
0001394920 has no ratios return
21
0001444754 has no ratios return
22
0001687316 has no ratios return
23
0001776738 has no ratios return
24
0000876320 has no ratios return
25
1
0001342423 fin ratios calculated
27
0001031896 has no ratios return
28
0001054476 has no ratios return
29
0001669400 has no ratios return
30
0000840759 has no ratios return
31
0000882483 has no ratios return
32
1
0001121702 fin 

/Users/j/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  



0001140868 has no ratios return
48
0001039213 has no ratios return
49
0001162142 has no ratios return
50
0000927536 has no ratios return
51
0001049486 has no ratios return
52
0001071917 has no ratios return
53
1
0001705843 fin ratios calculated
55
0000883906 has no ratios return
56
1
0001575858 fin ratios calculated
58
0000707600 has no ratios return
59
0001803705 has no ratios return
60
0001324087 has no ratios return
61
0001003515 has no ratios return
62
1
0001477246 fin ratios calculated
64
1
0000016160 fin ratios calculated
66
1
0001425292 fin ratios calculated
68
0001610462 has no ratios return
69
0000878944 has no ratios return
70
0001280396 has no ratios return
71
0001795139 has no ratios return
72
0000882991 has no ratios return
73
0001572819 has no ratios return
74
0001042327 has no ratios return
75
0000793931 has no ratios return
76
0000066895 has no ratios return
77
0000931612 has no ratios return
78
0001793341 has no ratios return
79
0000795255 has no ratios return
80
1
00

In [96]:
print(f"The list cik companies have connection with SEC website:")
for cik in valid_company_list:
    print(cik)


The list cik companies have connection with SEC website:
0000003545
0001756180
0001342423
0001121702
0001705843
0001575858
0001477246
0000016160
0001425292
0000037785
0001592016
0001159275
0000825542
0001482541
0000005981
0001441693
0000277638
0001302946
0001469443
0001285785
0000835011
0001133470
0001548240


In [97]:
len(valid_company_list)

23

In [88]:
for c in cik_list:
    if c == "0000005981":
        print(1)

1


In [40]:
print(f"The list cik companies have no connection with SEC website, no return data via XBRL:")
for cik in invalid_company_list:
    print(cik)

The list cik companies have no connection with SEC website, no return data via XBRL:
0000857949
0001321851
0001755672
0000941221
0001421461
0001725964
0001324404


In [97]:
df = read_fin_data_file(f'{sec_cfp_path}/agriculture_comprehensive/0001121702.csv')

In [98]:
dfr = Ratios_calculator(df)

In [99]:
dfr

,period.fiscal-year,period.end,revenue,net_income,operating_income,asset,current_asset,inventory,current_liability,debt,...,equity,eps,ROA1,ROE1,ROC1,OperatingMargin,NetMargin,CurrentRatio,QuickRatio,DERatio
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0.0,2020.0,1/1/21,799000.0,-10206000.0,-9609000.0,14522000.0,10342000.0,NaN,1814000.0,NaN,...,9532000.0,-4.30,-0.702796,-1.070709,NaN,-12.026283,-12.773467,5.701213,NaN,NaN
1.0,2019.0,1/1/20,806000.0,-12956000.0,-8596000.0,16718000.0,11684000.0,NaN,2207000.0,NaN,...,-4085000.0,-35.50,-0.774973,3.171603,NaN,-10.665012,-16.074442,5.294064,NaN,NaN
2.0,2018.0,1/1/19,556000.0,-9185000.0,-9319000.0,12960000.0,6377000.0,NaN,1641000.0,NaN,...,5698000.0,-36.99,-0.708719,-1.611969,NaN,-16.760791,-16.519784,3.886045,NaN,NaN
3.0,2017.0,1/1/18,944000.0,-9396000.0,-9283000.0,16882000.0,14917000.0,NaN,2375000.0,NaN,...,13502000.0,-3.29,-0.556569,-0.695897,NaN,-9.833686,-9.953390,6.280842,NaN,NaN
4.0,2016.0,1/1/17,1159000.0,-7604000.0,-10248000.0,10748000.0,7860000.0,0.0,2758000.0,NaN,...,6369000.0,-2.73,-0.707480,-1.193908,NaN,-8.842105,-6.560828,2.849891,2.849891,NaN
5.0,2015.0,1/1/16,1350000.0,-23681000.0,-12469000.0,17088000.0,14850000.0,51000.0,3910000.0,NaN,...,13028000.0,-0.95,-1.385826,-1.817700,NaN,-9.236296,-17.541481,3.797954,3.784910,NaN
6.0,2014.0,1/1/15,2800000.0,-29534000.0,-26829000.0,23135000.0,21965000.0,586000.0,4189000.0,NaN,...,18796000.0,-2.88,-1.276594,-1.571292,NaN,-9.581786,-10.547857,5.243495,5.103605,NaN
7.0,2013.0,1/1/14,5394000.0,-30506000.0,-28540000.0,26738000.0,25231000.0,1921000.0,6195000.0,NaN,...,20398000.0,-5.31,-1.140923,-1.495539,NaN,-5.291064,-5.655543,4.072801,3.762712,NaN
8.0,2012.0,1/1/13,42316000.0,3630000.0,3929000.0,53510000.0,48955000.0,3204000.0,5984000.0,NaN,...,47340000.0,0.11,0.067838,0.076679,NaN,0.092849,0.085783,8.180983,7.645555,NaN


# 7. SIC list aquisition (Get company list by industry)

In [23]:
url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&SIC=0100&owner=include&match=&start=&count=100&hidefilings=0'

In [46]:
url2 = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001443005&owner=include&count=40&hidefilings=0"

In [49]:
r1 = requests.get(url).content